In [1]:
import math 
import pandas as pd
import random 
import numpy as np
import pickle

In [2]:
# Setting simulation parameters values

N = [30, 50, 100, 500, 1000]  # population size
D = [0.01, 0.5, 1, 2] # Densities
T = [1, 2, 5, 7, 10] # infectious time in days

d = 2 # distance in meters 
iter = 100 # number of iterations
time = 1440 # exposure time in minutes


In [3]:
# import best model's trace 

with open(r"D:\Manal\Programming\project1\R0_simulation\trace_enveloppe_2m_0.1.pkl", 'rb') as f:
    trace_env = pickle.load(f)


p0_sample = trace_env.posterior["p0"].stack(sample=("chain", "draw")).values
p0_sample = np.random.choice(p0_sample, size=100)


lambda_sample = trace_env.posterior["lambda"].stack(sample=("chain", "draw")).values
lambda_sample = np.random.choice(lambda_sample, size=100)

In [4]:
%%time 

# Perform the simulations

data = []
results = dict()# using dictionary instead of  lists
for k in D: # density
    for n in N: # population size 
            for i in range(iter): # iterations
                p0 = p0_sample[i]
                lambda_ = lambda_sample[i]
                temp_results = []

                for m in range(n): # individuals
                    mu = np.random.uniform(0, 1) # a random number between 0 and 1
                    Rmax= np.sqrt(n/(k * np.pi)) # maximum radius distance
                    r0 = np.sqrt((n * mu)/(k * np.pi)) # generate initial radius distance r0
                    r = r0
                    p = 1 - (p0 * np.exp(-lambda_ * r0)) 
                    proba_minute = p
                    
                    for j in range(time):
                        if j == 0:
                            pass
                        else:
                            theta_ =  np.random.uniform(0, 2*np.pi) # a random number between 0 and pi
                            r = min(np.sqrt(r**2 + d**2 - 2 * r * d * np.cos(theta_) ),Rmax) # generate next radius distance per minute 
                            p = 1 - (p0 * np.exp(-lambda_ * r)) # probability per minute
                            proba_minute *= p # product on all probabilities per minute for each ID
                    
                    temp_results.append(proba_minute)

                for t in T:
                        proba_T = [1 - val**t for val in temp_results] # apply the exponent on infectious time
                        status = [np.random.binomial(1, val) for val in proba_T] # Bernoulli drawing for infectious status (0 or 1) using the final proba
                        a =[k, n, t] # creating the key for the dictionary
                        key = str(a) # writing as a string
                        try:
                            results[key].append(sum(status))
                        except KeyError:
                            results[key]=[sum(status)]
                        

CPU times: total: 1h 15min 2s
Wall time: 1h 15min 50s


In [8]:
with open(r"D:\Manal\Programming\project1\R0_simulation\results_dictionary.pkl", 'wb') as f:
    pickle.dump(results, f)

In [19]:
triplets = list(results.keys())

In [35]:
col_names = ['Density', 'Population_size', 'Infectious_time'] + [f'iter_{i}' for i in range(1, max(len(value) for value in results.values()) + 1)]



In [38]:
df = pd.DataFrame(
    [[float(triplet[1:-1].split(', ')[0])] + [int(x) for x in triplet[1:-1].split(', ')[1:]] + value for triplet, value in results.items()],
    columns=col_names
)

In [39]:
df

,Density,Population_size,Infectious_time,iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,...,iter_91,iter_92,iter_93,iter_94,iter_95,iter_96,iter_97,iter_98,iter_99,iter_100
0,0.01,30,1,2,2,2,5,4,1,2,...,1,3,1,1,1,3,2,3,1,4
1,0.01,30,2,5,4,4,6,9,5,2,...,6,1,7,5,7,4,3,5,4,4
2,0.01,30,5,11,8,7,10,15,6,4,...,10,9,10,10,10,6,3,12,6,13
3,0.01,30,7,8,9,13,11,19,11,7,...,8,11,6,16,14,6,14,10,9,10
4,0.01,30,10,18,14,14,18,17,10,12,...,15,11,20,12,16,11,17,13,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.00,1000,1,339,221,238,334,554,297,209,...,350,213,430,418,352,267,314,335,341,330
96,2.00,1000,2,576,394,493,565,797,514,359,...,532,425,626,662,576,458,530,565,551,552
97,2.00,1000,5,889,696,812,857,988,826,674,...,832,734,913,938,882,763,830,873,836,892
98,2.00,1000,7,950,808,889,942,996,910,789,...,912,832,966,977,955,870,930,936,940,943


In [ ]:
# Save the simulation results into a csv file

df.to_csv("simulation_results_v4.csv", index=False)